In [1]:
import sys
import os

# 获取当前工作目录
current_dir = os.getcwd()

# 添加项目根目录到 sys.path
project_dir = os.path.abspath(os.path.join(current_dir, './'))
# print(project_dir)
sys.path.append(project_dir)


In [2]:
import re
from typing import List
import numpy as np
import editdistance
from collections import defaultdict


# from pylatexenc.latexwalker import LatexWalker, 

from pylatexenc.latexwalker import LatexWalker
from sklearn.metrics import precision_score, recall_score, f1_score


In [3]:
def load_lns(fpath: str) -> List[str]:
    lns = []
    with open(fpath, "r") as f:
        for ln in f.readlines():
            lns.append(ln.strip())
    return lns

read test caption answer

In [4]:
test_dir = "../data/test"
test_caption_dir = os.path.join(test_dir,'caption.txt')
print(test_caption_dir)
caption_dict = {}

captions = load_lns(test_caption_dir)
max_caption_size = 0
count = 0
for cap in captions:
    img_name = cap.split()[0]
    caption_dict[img_name] = cap.split()[1:]
    if len(caption_dict[img_name]) > max_caption_size:
        max_caption_size = len(caption_dict[img_name])
    if len(caption_dict[img_name]) >= 600:
        count += 1
print(" max caption size ",max_caption_size)
print(count)
print(len(caption_dict.keys()),list(caption_dict.keys())[:5])      
print(caption_dict[list(caption_dict.keys())[0]])     

../data/test/caption.txt
 max caption size  823
7
3000 ['0054734', '0037174', '0031510', '0083407', '0037881']
['\\begin', '{aligned}', '\\boldsymbol', '{', 'K', '}', '=', '\\left', '(', '\\begin', '{array}', '{', 'c', 'c', 'c', '}', '0', '&', '0', '&', '\\frac', '{', '\\beta', '_', '{', '\\mathrm', '{', 'p', '}', '}', '}', '{', '\\gamma', '_', '{', '\\mathrm', '{', 'm', '}', '}', '}', '\\\\', '0', '&', '0', '&', '\\frac', '{', '\\beta', '_', '{', '\\mathrm', '{', 'm', '}', '}', '}', '{', '\\gamma', '_', '{', '\\mathrm', '{', 'm', '}', '}', '}', '\\\\', '\\mathcal', '{', 'P', '}', '\\frac', '{', '\\beta', '_', '{', '\\mathrm', '{', 'f', '}', '}', '}', '{', '\\gamma', '_', '{', '\\mathrm', '{', 'p', '}', '}', '}', '&', '\\frac', '{', '\\beta', '_', '{', '\\mathrm', '{', 'f', '}', '}', '}', '{', '\\gamma', '_', '{', '\\mathrm', '{', 'f', '}', '}', '}', '&', '0', '\\end', '{array}', '\\right', ')', '\\end', '{aligned}']


In [5]:
version = 78
ckp_folder = os.path.join("lightning_logs", f"version_{version}", "checkpoints")

In [6]:
pred_dir = '../result'
pred_files = os.listdir(pred_dir)
pred_dict = {}

for pred_file in pred_files:
    with open(os.path.join(pred_dir, pred_file), 'r',encoding='utf-8') as f:
            latex = f.readlines()[1].strip()
            if latex[0] == '$' and latex[-1] == '$':
                # replace start and end '$'
                latex = latex[1:-1]
            pred_dict[pred_file.split(".")[0]] = latex.split()
        
print(len(pred_dict.keys()),list(pred_dict.keys())[:5])

print(pred_dict[list(pred_dict.keys())[0]])

2993 ['0004028', '0109606', '0035325', '0097565', '0011280']
['\\begin', '{aligned}', 'm', '_', '{', 'l', '}', '=', '\\left', '(', '\\begin', '{array}', '{', 'c', 'c', 'c', '}', '0', '.', '0', '0', '2', '5', '6', '&', '-', '0', '.', '0', '1', '0', '5', '8', '&', '0', '\\\\', '-', '0', '.', '0', '1', '0', '5', '8', '&', '0', '.', '0', '4', '5', '9', '6', '&', '0', '\\\\', '0', '&', '0', '&', '1', '\\end', '{array}', '\\right', ')', 'm', '_', '{', '\\tau', '}', '\\end', '{aligned}']


In [7]:

def calculate_edit_distance(seq1, seq2):
    """计算两个Latex序列之间的编辑距离"""
    return editdistance.eval(seq1, seq2)


def calculate_accuracy(pred_dict, true_dict):
    """计算预测结果与真实结果之间的准确率"""
    correct_count = 0
    total_count = 0
    for key in pred_dict:
        if key in true_dict:
            pred_seq = pred_dict[key]
            true_seq = true_dict[key]
            if pred_seq == true_seq:
                correct_count += 1
            total_count += 1
    return correct_count / total_count if total_count > 0 else 0


def calculate_precision_recall_f1(pred_dict, true_dict):
    """计算精确度、召回率和F1分数"""
    all_precisions = []
    all_recalls = []
    all_f1s = []

    for key in pred_dict:
        if key in true_dict:
            pred_seq = pred_dict[key]
            true_seq = true_dict[key]
            # 转换为二进制分类
            true_labels = [1] * len(true_seq)
            pred_labels = [1 if i < len(pred_seq) and pred_seq[i] == true_seq[i] else 0 for i in range(len(true_seq))]
            if len(pred_labels) < len(true_labels):
                pred_labels.extend([0] * (len(true_labels) - len(pred_labels)))
            all_precisions.append(precision_score(true_labels, pred_labels))
            all_recalls.append(recall_score(true_labels, pred_labels))
            all_f1s.append(f1_score(true_labels, pred_labels))

    avg_precision = sum(all_precisions) / len(all_precisions) if all_precisions else 0
    avg_recall = sum(all_recalls) / len(all_recalls) if all_recalls else 0
    avg_f1 = sum(all_f1s) / len(all_f1s) if all_f1s else 0

    return avg_precision, avg_recall, avg_f1

def is_valid_latex(seq):
    """检查LaTeX序列是否合法"""
    latex_str = ''.join(seq)
    try:
        walker = LatexWalker(latex_str)
        nodes, pos, len_ = walker.get_latex_nodes()
        return True
    except ...:
        return False

def evaluate_by_length_intervals(pred_dict, true_dict, intervals):
    """按长度区间计算准确率、编辑距离和合法性"""
    interval_results = defaultdict(lambda: {
        'correct_count': 0, 
        'total_count': 0, 
        'total_edit_distance': 0,
        'invalid_count': 0
    })

    for key in pred_dict:
        if key in true_dict:
            pred_seq = pred_dict[key]
            true_seq = true_dict[key]
            length = len(true_seq)
            for start, end in intervals:
                if start <= length < end:
                    interval_results[(start, end)]['total_count'] += 1
                    interval_results[(start, end)]['total_edit_distance'] += calculate_edit_distance(pred_seq, true_seq)
                    if pred_seq == true_seq:
                        interval_results[(start, end)]['correct_count'] += 1
                    if not is_valid_latex(pred_seq):
                        interval_results[(start, end)]['invalid_count'] += 1
                    break

    results = {}
    for interval, data in interval_results.items():
        correct_count = data['correct_count']
        total_count = data['total_count']
        total_edit_distance = data['total_edit_distance']
        invalid_count = data['invalid_count']
        accuracy = correct_count / total_count if total_count > 0 else 0
        average_edit_distance = total_edit_distance / total_count if total_count > 0 else 0
        results[interval] = {
            'accuracy': accuracy,
            'average_edit_distance': average_edit_distance,
            'total_count': total_count,
            'invalid_count': invalid_count
        }

    return results


In [8]:
# 定义长度区间
intervals = [(0, 100), (100, 200), (200, 300), (300,400),(400,600),(600,700),(700,800),(800,900),(900,1000)]
true_dict = caption_dict


In [9]:

# 计算整体准确率
overall_accuracy = calculate_accuracy(pred_dict, caption_dict)
print(f'Overall Accuracy: {overall_accuracy}')

# 计算精确度、召回率和F1分数
avg_precision, avg_recall, avg_f1 = calculate_precision_recall_f1(pred_dict, true_dict)
print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')

# 计算各长度区间的指标
interval_results = evaluate_by_length_intervals(pred_dict, true_dict, intervals)

# 调试输出
print(f'Interval Results: {interval_results}')

for interval, metrics in interval_results.items():
    print(f'Interval {interval}: Accuracy = {metrics["accuracy"]}, Average Edit Distance = {metrics["average_edit_distance"]}, Total Count = {metrics["total_count"]}, Invalid Count = {metrics["invalid_count"]}')

Overall Accuracy: 0.46775810223855663
Average Precision: 1.0
Average Recall: 0.6930390607341295
Average F1 Score: 0.7392517982844312
Interval Results: {(0, 100): {'accuracy': 0.6376651982378855, 'average_edit_distance': 1.7797356828193833, 'total_count': 908, 'invalid_count': 0}, (100, 200): {'accuracy': 0.4791519434628975, 'average_edit_distance': 4.479151943462898, 'total_count': 1415, 'invalid_count': 0}, (200, 300): {'accuracy': 0.26053639846743293, 'average_edit_distance': 18.09961685823755, 'total_count': 522, 'invalid_count': 0}, (400, 600): {'accuracy': 0.0, 'average_edit_distance': 196.5, 'total_count': 36, 'invalid_count': 0}, (300, 400): {'accuracy': 0.0625, 'average_edit_distance': 53.651785714285715, 'total_count': 112, 'invalid_count': 0}}
Interval (0, 100): Accuracy = 0.6376651982378855, Average Edit Distance = 1.7797356828193833, Total Count = 908, Invalid Count = 0
Interval (100, 200): Accuracy = 0.4791519434628975, Average Edit Distance = 4.479151943462898, Total Coun

分析模型分析错误的原因，图片分辨率，过长？ 还是。

标准化。
测其他的指标